In [44]:
import pandas as pd
import os

In [47]:
new_df = []

for file in os.listdir('disability'):
    if file.endswith('csv'):
        data = pd.read_csv(f'disability/{file}')
        
        curr_year = int(file.split('.')[0].replace('ACSDT5Y', ''))
        data = data.T
        data.columns = data.iloc[0]
        data.drop(data.index[0], inplace=True)
        new_cols = [
            'Total',
            'Male Total',
            'Male Under 5 Years Total',
            'Male Under 5 Years With Disability',
            'Male Under 5 Years No Disability',
            'Male 5 to 17 Years Total',
            'Male 5 to 17 Years With Disability',
            'Male 5 to 17 Years No Disability',
            'Male 18 to 34 Years Total',
            'Male 18 to 34 Years With Disability',
            'Male 18 to 34 Years No Disability',
            'Male 35 to 64 Years Total',
            'Male 35 to 64 Years With Disability',
            'Male 35 to 64 Years No Disability',
            'Male 65 to 74 Years Total',
            'Male 65 to 74 Years With Disability',
            'Male 65 to 74 Years No Disability',
            'Male 75 Years and Over Total',
            'Male 75 Years and Over With Disability',
            'Male 75 Years and Over No Disability',
            'Female Total',
            'Female Under 5 Years Total',
            'Female Under 5 Years With Disability',
            'Female Under 5 Years No Disability',
            'Female 5 to 17 Years Total',
            'Female 5 to 17 Years With Disability',
            'Female 5 to 17 Years No Disability',
            'Female 18 to 34 Years Total',
            'Female 18 to 34 Years With Disability',
            'Female 18 to 34 Years No Disability',
            'Female 35 to 64 Years Total',
            'Female 35 to 64 Years With Disability',
            'Female 35 to 64 Years No Disability',
            'Female 65 to 74 Years Total',
            'Female 65 to 74 Years With Disability',
            'Female 65 to 74 Years No Disability',
            'Female 75 Years and Over Total',
            'Female 75 Years and Over With Disability',
            'Female 75 Years and Over No Disability',
        ]

        data.columns = new_cols

        temp_df = []
        for row in data.index:
            if 'Estimate' in row:
                new_row = row.split('!!')[0]
                temp_df.append(row)

        data.replace(',', '', regex=True, inplace=True)
        data = data.loc[temp_df]
        data.index = data.index.str.replace('!!Estimate', '')
        data = data.astype(int)

        data['Male Total Disability'] = data['Male Under 5 Years With Disability'] \
            + data['Male 5 to 17 Years With Disability'] \
            + data['Male 18 to 34 Years With Disability'] \
            + data['Male 35 to 64 Years With Disability'] \
            + data['Male 65 to 74 Years With Disability'] \
            + data['Male 75 Years and Over With Disability']

        data['Female Total Disability'] = data['Female Under 5 Years With Disability'] \
            + data['Female 5 to 17 Years With Disability'] \
            + data['Female 18 to 34 Years With Disability'] \
            + data['Female 35 to 64 Years With Disability'] \
            + data['Female 65 to 74 Years With Disability'] \
            + data['Female 75 Years and Over With Disability']

        data['Total Disability'] = data['Male Total Disability'] + data['Female Total Disability']
        
        stats = pd.DataFrame()
        stats['tract'] = data.index
        stats['total'] = data['Total'].fillna(0).values
        stats['male_percent'] = (data['Male Total'] / data['Total']).fillna(0).values
        stats['female_percent'] = (data['Female Total'] / data['Total']).fillna(0).values
        stats['disability_percent'] = (data['Total Disability'] / data['Total']).fillna(0).values
        stats['male_disability_percent'] = (data['Male Total Disability'] / data['Total']).fillna(0).values
        stats['female_disability_percent'] = (data['Female Total Disability'] / data['Total']).fillna(0).values
        stats['year'] = curr_year
        new_df.append(stats)

new_df = pd.concat(new_df).sort_values(by=['year', 'tract']).reset_index(drop=True)

In [48]:
new_df

,tract,total,male_percent,female_percent,disability_percent,male_disability_percent,female_disability_percent,year
0,"Census Tract 1005, Allegheny County, Pennsylvania",2321,0.506678,0.493322,0.203361,0.058165,0.145196,2014
1,"Census Tract 1011, Allegheny County, Pennsylvania",2815,0.482060,0.517940,0.141741,0.060746,0.080995,2014
2,"Census Tract 1014, Allegheny County, Pennsylvania",3326,0.478052,0.521948,0.141010,0.067348,0.073662,2014
3,"Census Tract 1016, Allegheny County, Pennsylvania",949,0.439410,0.560590,0.154900,0.081138,0.073762,2014
4,"Census Tract 1017, Allegheny County, Pennsylvania",1494,0.445783,0.554217,0.171352,0.063588,0.107764,2014
...,...,...,...,...,...,...,...,...
3589,Census Tract 9810; Allegheny County; Pennsylvania,0,0.000000,0.000000,0.000000,0.000000,0.000000,2022
3590,Census Tract 9811; Allegheny County; Pennsylvania,0,0.000000,0.000000,0.000000,0.000000,0.000000,2022
3591,Census Tract 9812; Allegheny County; Pennsylvania,0,0.000000,0.000000,0.000000,0.000000,0.000000,2022
3592,Census Tract 9818; Allegheny County; Pennsylvania,134,0.462687,0.537313,0.179104,0.097015,0.082090,2022


In [49]:
new_df.to_csv('total_disability.csv')